### SQL2Circuits development notebook

In [1]:
import os
import json
import numpy as np
from data_preparation.queries import QueryGenerator
from data_preparation.prepare import DataPreparation
from data_preparation.database import Database
from circuit_preparation.circuits import Circuits
from training.train import SQL2CircuitsEstimator
from training.sample_feature_preparation import SampleFeaturePreparator
from skopt import BayesSearchCV
from skopt.space import Real
import jax
jax.config.update("jax_enable_x64", True)

this_folder = os.path.abspath(os.getcwd())
seed_paths = ["data_preparation//query_seeds//JOB_query_seed_execution_time.json",
              "data_preparation//query_seeds//JOB_query_seed_cardinality.json"]
workload_types = ["execution_time", "cardinality"]
run_id = 1
ty = 1
workload_type = workload_types[ty]
database = Database("IMDB")
generator = QueryGenerator(run_id, workload_type = "cardinality", database = "IMDB", query_seed_file_path = seed_paths[ty])
query_file = generator.get_query_file()
data_preparator = DataPreparation(run_id, query_file, database = database, workload_type = workload_type, classification = 2)


Number of training queries is  398
Number of test queries is  140
Number of validation queries is  135
cardinality
Error while fetching data from PostgreSQL connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?



In [2]:
this_folder = os.path.abspath(os.getcwd())
output_folder = this_folder + "//circuit_preparation//data//circuits//" + str(run_id) + "//"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print("The new directory: ", output_folder, " is created.")

circuits = Circuits(run_id, query_file, output_folder, write_cfg_to_file = True, write_pregroup_to_file=True, generate_circuit_png_diagrams = True)
circuits.execute_full_transformation()

In [3]:
optimization_method = "Pennylane"
sf = SampleFeaturePreparator(run_id, data_preparator, circuits, "all", optimization_method)
X_train = sf.get_X_train()
X_valid = sf.get_X_valid()
y = sf.get_y()

opt = BayesSearchCV(
    SQL2CircuitsEstimator(run_id,
                        circuits = circuits,
                        workload = "cardinality",
                        classification = 2,
                        a = 0.01,
                        c = 0.01,
                        optimization_method = optimization_method,
                        epochs = 500), 
                        { 'a': Real(1e-4, 10, 'uniform'), 
                         'c': Real(1e-4, 10, 'uniform') }, 
                         n_iter = 20)

opt.fit(X_train, y, X_valid = X_valid)

results = dict(opt.cv_results_)
for key, value in results.items():
    if isinstance(value, np.ndarray):
        results[key] = value.tolist()
best_params = dict(opt.best_params_)
for key, value in best_params.items():
    if isinstance(value, np.ndarray):
        best_params[key] = value.tolist()
results["best_params"] = best_params
with open("training//results//" + str(run_id) + "//" + str(run_id) + "_cv_results_.json", "w") as f:
    json.dump(results, f)

id: 1
a: 0.01
c: 0.01
epochs: 500
classification: 4
workload: cardinality
optimization_medthod: Pennylane

id: 1
a: 0.01
c: 0.01
epochs: 500
classification: 4
workload: cardinality
optimization_medthod: Pennylane



Initializing new parameters
273
Number of training circuits:  309
Number of current validation circuits:  126


id: 1
a: 0.01
c: 0.01
epochs: 500
classification: 4
workload: cardinality
optimization_medthod: Pennylane

iters: 20
train/loss: 1315.9274
train/acc: 0.2427
valid/loss: 533.6241
valid/acc: 0.1825

iters: 40
train/loss: 1176.0453
train/acc: 0.2654
valid/loss: 445.5502
valid/acc: 0.3175

iters: 60
train/loss: 1264.1192
train/acc: 0.246
valid/loss: 486.9967
valid/acc: 0.254



initial_number_of_circuits = 20
number_of_circuits_to_add = 20
total_number_of_circuits = len(data_preparator.get_training_data_labels())

trainer = SQL2CircuitsEstimator(run_id, 
                              workload = "cardinality", 
                              a = 0.001, 
                              c = 0.001, 
                              classification = 2, 
                              optimization_method = "Pennylane")

#for i in range(initial_number_of_circuits, total_number_of_circuits, number_of_circuits_to_add):
#    sf = SampleFeaturePreparator(run_id, data_preparator, circuits, i, "pennylane")
#    X_train = sf.get_X_train()
#    X_valid = sf.get_X_valid()
#    y = sf.get_y()

    #trainer.fit_with_lambeq_noisyopt(X_train, y, X_valid = X_valid, save_parameters = True)
#    trainer.fit_with_pennylane_noisyopt(X_train, y, X_valid, save_parameters = True)
    #trainer.evaluate

# Train for the last time with all the data
sf = SampleFeaturePreparator(run_id, data_preparator, circuits, "all", "pennylane")
X_train = sf.get_X_train()
X_valid = sf.get_X_valid()
y = sf.get_y()

trainer.fit_with_pennylane_noisyopt(X_train, y, X_valid = X_valid, save_parameters = True)
